In [1]:
import skopt
from skopt.space import Real, Integer
import numpy as np
import pandas as pd
import sliceoptim.gcode as gc
import sliceoptim.core as core

In [2]:
printer = core.Printer(
            bed_size=[220, 220],
            nozzle_diameter=0.4,
            max_speed=120,
            min_speed=5
        )
filament = core.Filament(
            kind="pla",
            extrusion_temp_range=[180, 250],
            bed_temp_range=[25, 80],
            diameter=1.75,
        )

In [3]:
sample = gc.Sample('../assets/calicat.stl', is_first_layer=False, printer=printer, filament=filament)

In [4]:
gcode = sample.gen_gcode()

In [12]:
# parse lines
# handle G1 X Y / F7
lines = gcode.splitlines()

In [15]:
l1 = lines[150]
l1

'G1 X122.911 Y114.003 E2.14805'

In [24]:
def parse_line(line):
    move = {"G": None, "X": None, "Y": None, "Z": None, "F": None}
    for key, val in move.items():
        if key in line:
            d = line.split(key)[1].split(' ')[0]
            try:
                if key == "G":
                    move[key] = int(d)
                else:
                    move[key] = float(d)
            except ValueError:
                continue
    return move

In [30]:
minutes = 0
feedrate = 0
last_state = {"G": 0, "X": 0, "Y": 0, "Z": 0, "F": 0}
for l in lines:
    move = parse_line(l)
    distance = 0
    if move["G"] == 1:
        if move["F"] is not None:
            feedrate = move["F"]
            last_state["F"] = move["F"]
        for c in ["X", "Y", "Z"]:
            if move[c] is not None and move[c] != last_state[c]:
                distance = np.abs(move[c] - last_state[c])
                last_state[c] = move[c]
    if feedrate != 0 and distance !=0:
        minutes += distance / feedrate

print(minutes)
        

20.75176051282052
